In [3]:
import pandas as pd
import ast
from collections import Counter
import math
import re

In [4]:
def get_bigram_corpus(wordlist):
    """
    Returns the bigram corpus given by the input [wordlist]
    
    get_bigram_corpus creates a dictionary with bigrams from the [wordlist]
    as keys and counts the instances of each bigram to assign values (except
    for the bigram ('.', '<s>') which represents the start of one review and
    the end of another)
    
    wordlist: a list of words (strings)
    """
    corpus = {}
    for i, word in enumerate(wordlist[1:], start=1):
        if word != '<s>':
            if (wordlist[i-1], word) not in corpus:
                corpus[(wordlist[i-1], word)] = 1
            else:
                corpus[(wordlist[i-1], word)] += 1
    return corpus

def get_smooth_bigram_corpus(tokenlist, bigram_corpus):
    """
    Returns a dataframe object where the columns and rows are labeled with the tokens
    in [tokenlist] and the elements are the counts of each bigram (defaulting to 1 to handle
    add-one smoothing) in the format (row, column) so that the count for bigram (x, y) is 
    found with df.loc[x, by]
    
    get_smooth_bigram_corpus also appends the unknown word character [<UNK>] to handle 
    unknown words
    
    tokenlist: a list of tokens (strings)
    bigram_corpus: a dictionary of bigram:count pairings
    """
    tokenlist.append('<UNK>')
    df = pd.DataFrame(1, index = tokenlist, columns = tokenlist) 
    for bigram in bigram_corpus:
        df.loc[bigram[0], bigram[1]] += bigram_corpus[bigram]
    return df

def get_smooth_bigram_prob(bigram, smooth_bigram_corpus):
    """
    Returns the probability of a given [bigram] on a given [smooth_bigram_corpus]
    
    get_smooth_bigram_prob takes the ratio of the value of [bigram] (df.loc(bigram[0], bigram[1]))
    in the table [smooth_bigram_corpus] to the sum of all elements in the same row
    
    bigram: a bigram (tuple of strings)
    smooth_bigram_corpus: a dataframe with tokens as row and column names and bigram counts as values
    """
    return smooth_bigram_corpus.loc[bigram[0], bigram[1]] / smooth_bigram_corpus.loc[bigram[0]].sum()

In [5]:
train_df = pd.read_csv('./data_release/train.csv', encoding='cp1252')


In [6]:

for x in train_df.iterrows():
    print(x[1][0])
    break

Ca n't fail to be entertaining .


In [7]:
train_df

,sentence,pos_seq,label_seq
0,Ca n't fail to be entertaining .,"['VERB', 'ADV', 'VERB', 'PART', 'VERB', 'ADJ',...","[0, 0, 0, 0, 0, 0, 0]"
1,How much was he going to tell her ?,"['ADV', 'ADJ', 'VERB', 'PRON', 'VERB', 'PART',...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"Up until that news hit the Committee , Don had...","['ADP', 'ADP', 'DET', 'NOUN', 'VERB', 'DET', '...","[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,Could go on to the rugby and go with them coul...,"['VERB', 'VERB', 'PART', 'ADP', 'DET', 'NOUN',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"Finally , we went to the office and they gave ...","['ADV', 'PUNCT', 'PRON', 'VERB', 'ADP', 'DET',...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
5,It must be shown that the defendant intended (...,"['PRON', 'VERB', 'VERB', 'VERB', 'ADP', 'DET',...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
6,The Englishman nodded and poured himself more ...,"['DET', 'PROPN', 'VERB', 'CCONJ', 'VERB', 'PRO...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,I demanded .,"['PRON', 'VERB', 'PUNCT']","[0, 0, 0]"
8,What is not known is information on the locati...,"['NOUN', 'VERB', 'ADV', 'VERB', 'VERB', 'NOUN'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,That 's getting in the,"['DET', 'VERB', 'VERB', 'ADP', 'DET']","[1, 0, 0, 0, 0]"


# HMM Model

In [8]:
class HMM():
    def __init__(self, training_df):
        self.bigram_counts = {}
        self.POS_counts = Counter()
        
        self.lex_gen = {}
        self.word_pattern = re.compile("(\w+|<s> |[,.!?;\(\)])")

        self.training_df = training_df
            
                for row in self.training_df.iterrows():
            
            POS_list = ast.literal_eval(row[1][1])
            POS_bigram_counts = get_bigram_corpus(POS_list)
            
            sentence_list = row[1][0].split()
            
            for i, word in enumerate(sentence_list):
                if word not in self.lex_gen:
                    self.lex_gen[word] = {POS_list[i] : 1}
                else:
                    if POS_list[i] not in self.lex_gen[word]:
                        self.lex_gen[word][POS_list[i]] = 1
                    else:
                        self.lex_gen[word][POS_list[i]] += 1
            
            # Updates list of POS counts across whole training set 
            self.POS_counts.update(POS_list)
            
            for bigram in POS_bigram_counts: 
                if bigram in self.bigram_counts:
                    self.bigram_counts[bigram] += POS_bigram_counts[bigram]
                else:
                    self.bigram_counts[bigram] = POS_bigram_counts[bigram]
        
        self.smoothed_bigram_counts_df = get_smooth_bigram_corpus(list(self.POS_counts), self.bigram_counts)
    


      
    def prob_tagged_sequence(self, sequence):
        '''
        
        sequence: tuple where first element is sentence as a string, second element is a 
            list of POS associated with each word of sentence 
        '''
        
        sentence_list = self.word_pattern.findall(sequence[0])
        POS_list = sequence[1]
        
        log_prob_acc = 0

        for i, POS in enumerate(POS_list):
            if i == 0:
                continue

            POS_bigram = (POS_list[i-1], POS) 
            log_prob_acc += math.log(get_smooth_bigram_prob(POS_bigram, self.smoothed_bigram_counts_df))

            log_prob_acc += math.log(self.lex_gen[sentence_list[i]].get(POS, 1)/dict(self.POS_counts)[POS]) 
        
        return math.exp(log_prob_acc)
        
        
        
    

In [14]:
hmm = HMM(train_df)
# print(hmm.POS_counts)
hmm.prob_tagged_sequence(('the dog ', ['DET','NOUN']))

hmm.smoothed_bigram_counts_df

,VERB,ADV,PART,ADJ,PUNCT,PRON,ADP,DET,NOUN,PROPN,CCONJ,NUM,INTJ,X,SYM,<UNK>
VERB,3683,2792,1377,1580,1643,1654,3067,2713,1372,358,252,234,71,1,1,1
ADV,2131,734,95,834,1095,501,766,396,111,57,122,68,19,1,1,1
PART,1736,52,22,108,213,9,204,98,395,48,47,15,5,1,1,1
ADJ,530,127,216,769,729,99,705,207,5721,174,309,61,6,3,1,1
PUNCT,1054,570,40,515,394,806,781,681,506,536,753,278,103,5,1,1
PRON,5089,377,133,48,689,46,243,137,59,8,69,9,14,2,1,1
ADP,498,287,26,1954,165,1064,293,4925,2372,1213,47,413,17,5,1,1
DET,558,216,23,2508,160,21,136,20,6104,858,21,145,12,2,1,1
NOUN,3292,649,607,494,5430,473,5662,239,2096,112,1249,77,15,3,3,1
PROPN,1002,104,364,54,1406,40,422,31,332,1484,319,125,9,4,3,1


In [10]:
k = '["hello", "hi"]'

In [11]:
y = ast.literal_eval(k)

In [12]:
hmm

In [13]:
diction = {'hello' : 1, 'apple' : 2}